In [32]:
import pickle
import numpy as np
import itertools
import time
import torch
import os
import segutil

In [1]:
import tiktoken
gptenc = tiktoken.get_encoding('gpt2')

In [2]:
sent = "First Citizen:\nBefore we proceed any further, hear me speak."

In [5]:
from bertviz import model_view

In [6]:
import model

In [53]:
import importlib
importlib.reload(model)

<module 'model' from '/Users/rastislavhronsky/ML-research/nanoGPT/model.py'>

In [54]:
class Config:
    vocab_size=10
    block_size=9
    n_layer = 6
    n_head = 8
    n_embd = 128
    dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
    bias = False # do we use bias inside LayerNorm and Linear layers?
    force_no_flash=True
config = Config()

block = model.GPT(config)
x = torch.randint(0,config.vocab_size,(16,config.block_size))

number of parameters: 1.18M


In [55]:
logits, att, loss = block(x)

In [60]:
att[0,0,0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5264, 0.4736, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3273, 0.3516, 0.3211, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2362, 0.2652, 0.2504, 0.2481, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1882, 0.2037, 0.2002, 0.2084, 0.1995, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1478, 0.1771, 0.1561, 0.1727, 0.1715, 0.1746, 0.0000, 0.0000, 0.0000],
        [0.1369, 0.1504, 0.1396, 0.1431, 0.1351, 0.1451, 0.1498, 0.0000, 0.0000],
        [0.1095, 0.1381, 0.1291, 0.1306, 0.1253, 0.1350, 0.1181, 0.1143, 0.0000],
        [0.1068, 0.1115, 0.1070, 0.1133, 0.1151, 0.1173, 0.1072, 0.1122, 0.1095]])

In [17]:
import torch
hasattr(torch.nn.functional, 'scaled_dot_product_attention')

True

In [63]:
list(map(str, range(10)))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [67]:
att[:1, 0].shape

torch.Size([1, 8, 9, 9])

In [70]:
model_view([att[:1, 0]], tokens=list(map(str, range(9))))

<IPython.core.display.Javascript object>